# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [ ]:
# Add any additional import statements you need here




In [2]:
# change `data_location` to the location of the folder on your machine.
data_location = "/Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

cher= "cher/"
robyn= "robyn/"

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [2]:
# Read in the lyrics data

import os
from collections import defaultdict

# change `data_location` to the location of the folder on your machine.
data_location = "/Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

cher = "cher/"
robyn = "robyn/"

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

# Function to read files from a specific folder
def read_files_from_folder(folder_path):
    files_content = {}
    print(f"Reading files from folder: {folder_path}")  # Debug print
    for root, dirs, files in os.walk(folder_path):
        print(f"Found directory: {root}")  # Debug print
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                print(f"Reading file: {file_path}")  # Debug print
                with open(file_path, 'r', encoding='utf-8') as f:
                    files_content[file] = f.read()
    return files_content

# Read the lyrics data
lyrics_path = os.path.join(data_location, lyrics_folder)
cher_path = os.path.join(lyrics_path, cher)
robyn_path = os.path.join(lyrics_path, robyn)

lyrics_data = {}
lyrics_data['cher'] = read_files_from_folder(cher_path)
lyrics_data['robyn'] = read_files_from_folder(robyn_path)

# Organize the lyrics data into a dictionary with artist and song as keys
lyrics_dict = defaultdict(dict)
for artist in lyrics_data:
    for file, content in lyrics_data[artist].items():
        song = file.replace('.txt', '')
        lyrics_dict[artist][song] = content

# Read the Twitter data
twitter_path = os.path.join(data_location, twitter_folder)
twitter_data = read_files_from_folder(twitter_path)

# Organize Twitter descriptions into a dictionary
twitter_dict = defaultdict(list)
for file, content in twitter_data.items():
    artist = file.replace('.txt', '')
    descriptions = content.split('\n')
    twitter_dict[artist] = descriptions

# Output the dictionaries
print("\nLyrics Dictionary:")
count(lyrics_dict)

print("\nTwitter Dictionary:")
count(twitter_dict)

Reading files from folder: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/
Found directory: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/
Reading file: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/cher_comeandstaywithme.txt
Reading file: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/cher_pirate.txt
Reading file: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/cher_stars.txt
Reading file: /Users/jesse/Desktop/School/University of San Diego/ADS 509- Applied Text Mining/Module 2/ads-tm-token-norm/M1 Results/lyrics/cher/cher_thesedays.txt
Reading file: /Users/jesse/Desktop/

In [ ]:
# Read in the twitter data

In [ ]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.

## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [ ]:
# your code here

### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: For your second artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. You do not need to calculate sentiment on non-emoji content for this section.

In [ ]:
# your code here

Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

